In [1]:
!pip install datasets
!pip install accelerate
!pip install -U bitsandbytes
!pip install transformers
!pip install peft
!python -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm
from peft import LoraConfig, get_peft_model
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from peft import AutoPeftModelForCausalLM, PeftConfig

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
peft_model_id = "yujinjin/david-gemma-finetuned-4400" ##모델 이름 수정
config = PeftConfig.from_pretrained(peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)
model = AutoPeftModelForCausalLM.from_pretrained(
        peft_model_id,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
)

adapter_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

In [10]:
def remove_newlines(text):
    return text.replace('\n', '')

def remove_double(text):
    return text.replace('  ', '')

def remove_noise(text):
    return text.replace('#', '')

def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}

In [40]:
def generate_response(context, question, model, tokenizer):
    prompt = f"너는 주어진 문맥을 토대로 질문에 대해 간결하게 답변하는 챗봇이야.\
주어진 6개의 단계를 따라 질문에 대한 답변을 해야해.\
단계 1. 문맥에서 질문에 대한 내용이 있는 부분을 찾아줘.\
단계 2. 질문에서 물어보는 것이 뭔지 찾아줘.\
단계 3. 단계1에서 찾은 부분에서 질문에 대한 답변만 찾아줘.\
단계 4. 문맥에 있는 답변을 바꾸거나 변형하지 말고 끝부분만 다듬어줘.\
단계 5. 답변이 10글자 이내라면 간단하게 키워드로 알려줘.\
단계 6. 답변이라 생각한 근거를 알려줘\
질문에 대한 틀린 답변과 옳은 답변의 예시를 알려줄게.\
질문: 큐알아트의 방문이 가능한 매장이 있는 곳이 어디지\
틀린 답변: 3호선 마두역 인근\
옳은 답변: 마두역\
\
질문: 풍력발전 장치는 얼마마다 점검 받는 것으로 바뀌었어\
틀린 답변: 4년에서 2년으로 단축\
옳은 답변: 2년\
\
질문: 새만금개발청은 케이팝 축제를 개최할 예정인 건 언제까지야\
틀린 답변: 9월 10일부터 12일까지\
옳은 답변: 12일\
\
질문: 사진 부문 대상 상은 무엇인가\
틀린 답변: 산림청장상과 상금 100만원\
옳은 답변: 산림청장상\
\
질문: 일품양평해장국에서 창업자를 대상으로 2월에 무슨 행사를 하고 있어\
틀린 답변: 2월 한 달 동안 업종변경 창업 시 양평해장국세트 150인분을 본사에서\
옳은 답변: 양평해장국세트 150인분을 본사에서 무상으로 제공\
\
질문: 강원도에서 점차 소비자심리지수가 상향되고 있는 이유가 뭐야\
틀린 답변: 1분기 경제 성장률이 예상치를 웃돌았고 수출 호조, 최근 백신 접종\
옳은 답변: 예상치를 웃돈 경기 성장률과 수출 호조, 백신 접종\
\
질문: 미래에셋투자와 연금센터 대표는 왜 은퇴 후 자산관리가 중요하다고 강조했어\
틀린 답변: 본격적인 고령화 시대가 도래 하면서 은퇴고객의 자산관리 중요성이 날로\
옳은 답변: 본격적인 고령화 시대가 도래\
\
질문: 누가 핵심 목표를 담은 신년사를 발표했어\
틀린 답변: 구자열 LS그룹 회장\
옳은 답변: 구자열\
\
질문: 도지코인이 시장 흐름과 다르게 급격히 상승하는 이유는 뭐야\
틀린 답변: 저가 매수세가 유입됐기 때문\
옳은 답변: 저가 매수세가 유입\
\
질문: CES 2021에 참가한 분야 중 하나는\
틀린 답변: 건강·웰니스, 5세대(5G)·사물인터넷, 로봇·인공지능\
옳은 답변: 건강·웰니스\
\
질문: 어피너티 측의 주가 산출을 의뢰받은 회사\
틀린 답변: 딜로이트 안진회계법인(안진)\
옳은 답변: 안진\
\
문맥: {context}\n 질문: {question}\n  ##옳은 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=70, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.6, top_p=0.95, top_k=50, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "##옳은 답변:" in response:
        response = response.split("##옳은 답변:", 1)[1][:40]
        if "질문" in response:
            response = response.split("질문", 1)[0]
        if "\n" in response:
            response = response.replace("\n", "")
        if "옳은" in response:
            response = response.split("옳은", 1)[0]
        if "근거" in response:
            response = response.split("근거", 1)[0]
        if "답변" in response:
            response = response.split("답변", 1)[0]
        if "키워드" in response:
            response = response.split("키워드", 1)[0]
        if "답변 근거" in response:
            response = response.split("답변 근거", 1)[0]
        if "틀린 답변" in response:
            response = response.split("틀린 답변", 1)[0]
        if "#틀린" in response:
            response = response.split("#틀린", 1)[0]
        if "#" in response:
            response = response.replace("#", "")[0]
    return response.strip()

In [41]:
def main():
    test_data = pd.read_csv('./test.csv')
    test_data['context'] = test_data['context'].apply(remove_newlines)
    test_data['context'] = test_data['context'].apply(remove_double)
    test_data['context'] = test_data['context'].apply(remove_noise)

    submission_dict = {}

    for index, row in test_data.iterrows():
        try:
            context = row['context']
            question = row['question']
            id = row['id']

            if context is not None and question is not None:
                answer = generate_response(context, question, model, tokenizer)
                submission_dict[id] = answer

                # 질문과답변 출력
                print(f"id: {id}")
                print(f"질문: {question}")
                print(f"답변: {answer}")
                print("-" * 50)
            else:
                submission_dict[id] = 'Invalid question or context'
                print(f"Invalid question or context for id: {id}")
                print("-" * 50)

        except Exception as e:
            print(f"Error processing question {id}: {e}")
            print("-" * 50)

    df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
    df.to_csv( './david_prompt_ver4_submission.csv', index=False)

In [ ]:
main()

id: TEST_0000
질문: 어떤 기관이 지역 중소기업과 소상공인들에게 큰 힘이 되는 통상진흥원인가요
답변: 제주특별자치도 경제통상진흥원
--------------------------------------------------
id: TEST_0001
질문: 제주 경제통상진흥원이 어떤 해외 시장을 우선적으로 공략하고 있나요
답변: 중국과 일본
--------------------------------------------------
id: TEST_0002
질문: 상해대표처는 어느 시점에 설립되었습니까
답변: 2015년 8월
--------------------------------------------------
id: TEST_0003
질문: 동경통상대표부는 어느 지역에 설립되었습니까
답변: 도쿄 신주쿠
--------------------------------------------------
id: TEST_0004
질문: 제주경제통상진흥원이 지난해에 도내에서 생산된 제품에 대한 일본 수출에 얼마를 지원했나요
답변: 31억7200만원
--------------------------------------------------
id: TEST_0005
질문: 서경대 실용음악과 보컬전공의 대입 시험은 어디에서 진행되었나
답변: 운동장
--------------------------------------------------
id: TEST_0006
질문: 천막 고사장으로 변경한 이유는 무엇인가
답변: 코로나19
--------------------------------------------------
id: TEST_0007
질문: 2021학년도 서경대 실용음악과 보컬 전공 수시에 몇 명이 지원했나
답변: 1710명
--------------------------------------------------
id: TEST_0008
질문: 하나의 시험 당 약 몇 명이 참가했나
답변: 25명
-------------------------------------